In [2]:
import sqlite3
import csv
from datetime import datetime

# 创建数据库和表
def create_tables(conn):
    conn.execute('''CREATE TABLE IF NOT EXISTS apple_stock (
                    date TEXT PRIMARY KEY,
                    close REAL)''')
    
    conn.execute('''CREATE TABLE IF NOT EXISTS google_stock (
                    date TEXT PRIMARY KEY,
                    close REAL)''')
    
    conn.execute('''CREATE TABLE IF NOT EXISTS tesla_stock (
                    date TEXT PRIMARY KEY,
                    close REAL)''')

# 导入Apple数据
def import_apple_data(conn, filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # 跳过标题行
        for row in reader:
            date = row[0]
            close = float(row[4])
            conn.execute("INSERT OR IGNORE INTO apple_stock (date, close) VALUES (?, ?)", 
                         (date, close))

# 导入Google数据
def import_google_data(conn, filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # 跳过标题行
        for row in reader:
            date = row[0]
            close = float(row[4])
            conn.execute("INSERT OR IGNORE INTO google_stock (date, close) VALUES (?, ?)", 
                         (date, close))

# 导入Tesla数据
def import_tesla_data(conn, filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # 跳过标题行
        for row in reader:
            # 提取日期部分（去掉时区信息）
            full_date = row[0]
            date_obj = datetime.strptime(full_date.split('+')[0].strip(), '%Y-%m-%d %H:%M:%S')
            date = date_obj.strftime('%Y-%m-%d')
            close = float(row[2])  # Close列是第三列
            conn.execute("INSERT OR IGNORE INTO tesla_stock (date, close) VALUES (?, ?)", 
                         (date, close))

# 计算金叉位置
def calculate_golden_cross(conn, table_name):
    # 使用窗口函数计算移动平均
    query = f"""
    WITH moving_averages AS (
        SELECT 
            date,
            close,
            AVG(close) OVER (ORDER BY date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS ma5,
            AVG(close) OVER (ORDER BY date ROWS BETWEEN 9 PRECEDING AND CURRENT ROW) AS ma10
        FROM {table_name}
    ),
    cross_signals AS (
        SELECT 
            date,
            close,
            ma5,
            ma10,
            LAG(ma5) OVER (ORDER BY date) AS prev_ma5,
            LAG(ma10) OVER (ORDER BY date) AS prev_ma10
        FROM moving_averages
    )
    SELECT date, close
    FROM cross_signals
    WHERE ma5 > ma10 AND (prev_ma5 < prev_ma10 OR prev_ma5 IS NULL)
    ORDER BY date
    """
    return conn.execute(query).fetchall()

# 主程序
def run():
    # 初始化数据库
    conn = sqlite3.connect('stock_data.db')
    create_tables(conn)
    
    # 导入数据
    import_apple_data(conn, 'Apple Dataset.csv')
    import_google_data(conn, 'GOOGL.csv')
    import_tesla_data(conn, 'Tesla.csv')
    conn.commit()
    
    # 计算并显示金叉
    results = {}
    results['Apple'] = calculate_golden_cross(conn, 'apple_stock')
    results['Google'] = calculate_golden_cross(conn, 'google_stock')
    results['Tesla'] = calculate_golden_cross(conn, 'tesla_stock')
    
    # 打印结果
    for company, crosses in results.items():
        print(f"\n{company}金叉出现的位置（日期）：")
        for date, close in crosses:
            print(f"日期: {date}, 收盘价: {close:.2f}")
    
    conn.close()

In [3]:
run()


Apple金叉出现的位置（日期）：
日期: 1980-12-23, 收盘价: 0.14
日期: 1981-01-20, 收盘价: 0.14
日期: 1981-03-03, 收盘价: 0.12
日期: 1981-03-19, 收盘价: 0.11
日期: 1981-04-07, 收盘价: 0.11
日期: 1981-04-24, 收盘价: 0.13
日期: 1981-05-20, 收盘价: 0.13
日期: 1981-06-15, 收盘价: 0.14
日期: 1981-07-20, 收盘价: 0.11
日期: 1981-07-30, 收盘价: 0.11
日期: 1981-09-02, 收盘价: 0.10
日期: 1981-10-05, 收盘价: 0.08
日期: 1981-10-21, 收盘价: 0.09
日期: 1981-11-17, 收盘价: 0.08
日期: 1981-11-23, 收盘价: 0.08
日期: 1981-11-27, 收盘价: 0.08
日期: 1981-12-02, 收盘价: 0.08
日期: 1981-12-17, 收盘价: 0.09
日期: 1982-01-19, 收盘价: 0.09
日期: 1982-02-03, 收盘价: 0.09
日期: 1982-02-22, 收盘价: 0.08
日期: 1982-03-23, 收盘价: 0.08
日期: 1982-04-02, 收盘价: 0.08
日期: 1982-05-06, 收盘价: 0.07
日期: 1982-06-16, 收盘价: 0.06
日期: 1982-07-15, 收盘价: 0.06
日期: 1982-08-13, 收盘价: 0.06
日期: 1982-10-04, 收盘价: 0.08
日期: 1982-11-01, 收盘价: 0.12
日期: 1982-12-01, 收盘价: 0.15
日期: 1982-12-22, 收盘价: 0.14
日期: 1983-01-14, 收盘价: 0.15
日期: 1983-03-21, 收盘价: 0.20
日期: 1983-03-30, 收盘价: 0.20
日期: 1983-04-14, 收盘价: 0.20
日期: 1983-05-05, 收盘价: 0.24
日期: 1983-05-23, 收盘价: 0.26
日期: 1983-09-01, 收盘价